In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
!pip install tiktoken

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [6]:
!mkdir pdfs

In [4]:
loader = PyPDFDirectoryLoader("pdfs")

In [5]:
data=loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [10]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
text_chunks

In [ ]:
print(text_chunks[0].page_content)

In [ ]:
print(text_chunks[1].page_content)

In [ ]:
print(text_chunks[2].page_content)

In [ ]:
len(text_chunks)

In [ ]:
print(text_chunks[89].page_content)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [39]:
import os
KEY=os.getenv("OPENAI_API_KEY")

In [40]:
# print(KEY)

In [41]:
embedding=OpenAIEmbeddings()

In [ ]:
embedding.embed_query("How are you")

In [43]:
pcKEY=os.getenv("PINECONE_API_KEY")
pcENV=os.getenv("PINECONE_API_ENV")

In [ ]:
# print(pcKEY)
# print(pcENV)

In [67]:
# !pip install "pinecone-client[grpc]"

In [68]:
from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec

In [69]:
pc=Pinecone(api_key=pcKEY)

In [75]:
pcIDX=os.getenv("PINECONE_IDX_NAME")

In [ ]:
print(pcIDX)

In [77]:
pcIDX="index"

In [78]:
index=pc.Index(pcIDX)

In [82]:
from langchain.vectorstores import Pinecone

### from_texts mathod is not present in pinecone directly but you can use it if you import pinecone using langchain

In [ ]:
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name=pcIDX)

In [83]:
query="Attention all you need"

In [ ]:
docs=docsearch.similarity_search(query)

In [ ]:
llm=OpenAI()

In [ ]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [87]:
query="Attention all you need"

In [ ]:
qa.run(query)

## QA System

In [ ]:
import sys
while True:
    user_input = input(f"Input Prompt: ")
    if user_input == 'exit':
        print('exiting')
        sys.exit()
    if user_input == '':
        continue
    result = qa({'query': user_input})
    print(f"Answer: {result['result']}")